## Simulation script for exploration of missing data strategies

In this script, we perform simulations to explore the effect of several missing data techniques on the outcome of a regression analysis. We work with two real, example datasets (imported with file: import_real_data.ipynb) and four custom datasets (created in the file: generate_data.ipynb). We c an amputation function and generated multiple forms of missing data. We deal with the missing data by the following missing data methods: drop, mean imputation, median imputation, regression imputation, stochastic regression imputation and random imputation. 

In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma
from scipy import sparse
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# for performing simulation
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import explained_variance_score as ev
from math import sqrt

# for creating imputation functions in class Imputer
from sklearn.preprocessing.imputation import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.utils.sparsefuncs import _get_median
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import FLOAT_DTYPES
from sklearn.externals import six

import matplotlib.pyplot as plt
%matplotlib inline

## Amputation function

In [2]:
# Function to delete entries of dataframe
def delete_data(df_,mechanism,output_variable,missing_row_proportion=0.05):
    
    df = df_.copy()
    index_ = df.index.values
    all_columns = df.drop(output_variable,1).columns.tolist()
    total_rows = int(len(df)*missing_row_proportion)
    cells_per_column = int(total_rows / 2) #50% of cells in selected rows are made missing    
    df['z'] = df[output_variable] + 2*np.random.randn(len(df))
 
    if mechanism == 'MCAR': 
        
        index_selected = np.random.choice(index_,total_rows,replace=False)           
        for col in all_columns:
            drop_index = np.random.choice(index_selected,cells_per_column,replace=False)
            df[col].loc[drop_index] = None
            
    elif mechanism == 'MARZ':
        
        weights = df.loc[index_]['z'].values + abs(df.loc[index_]['z'].values.min()) + 0.05
        weights = weights/weights.sum(axis=0)
        index_selected = np.random.choice(index_,total_rows,p=weights,replace=False) 
        
        #records with a large y value receive a large weight
        weights = df.loc[index_selected]['z'].values+\
        abs(df.loc[index_selected]['z'].values.min()) + 0.05
        weights = weights/weights.sum(axis=0)
        for col in all_columns:
            drop_index = np.random.choice(index_selected,size=cells_per_column,p=weights,replace=False)            
            df[col].loc[drop_index] = None
            
    elif mechanism == 'MARX':
        
        index_selected = np.random.choice(index_,total_rows,replace=False) 
                
        weights = []
        for col in all_columns:            
            ind = all_columns.index(col) + 1
            if ind == len(all_columns):
                ind = 0
            x_col = all_columns[ind]
            
            #records with a large value on the next X feature receive a large weight for the current X feature
            wgts = df.loc[index_selected][x_col].values+\
            abs(df.loc[index_selected][x_col].values.min()) + 0.05
            wgts = wgts/wgts.sum(axis=0)
            weights.append(wgts) 
        
        for col in all_columns:
            ind = all_columns.index(col)
            drop_index = np.random.choice(index_selected,size=cells_per_column,p=weights[ind],replace=False)
            df[col].loc[drop_index] = None
            
    elif mechanism == 'MNARZ':
        
        weights = df.loc[index_]['z'].values + abs(df.loc[index_]['z'].values.min()) + 0.05
        weights = weights/weights.sum(axis=0)
        index_selected = np.random.choice(index_,total_rows,p=weights,replace=False)
                
        #records with a large y value receive a large weight
        weights = df.loc[index_selected]['z'].values+\
        abs(df.loc[index_selected]['z'].values.min()) + 0.05
        weights = weights/weights.sum(axis=0) 
        for col in all_columns:
            drop_index = np.random.choice(index_selected,size=cells_per_column,p=weights,replace=False)
            df[col].loc[drop_index] = None
                                                              
    elif mechanism == 'MNARX':   
        
        index_selected = np.random.choice(index_,total_rows,replace=False) 

        for col in all_columns:
            #records with a large value on a feature receive a large weight
            weights = df.loc[index_selected][col].values+\
            abs(df.loc[index_selected][col].values.min()) + 0.05
            weights = weights/weights.sum(axis=0) 
            drop_index = np.random.choice(index_selected,size=cells_per_column,p=weights,replace=False)
            df[col].loc[drop_index] = None

    return df

# Missing data strategies

In [3]:
def apply_train_test(data,test_size=0.4):
    train, test = train_test_split(data.index.values,test_size=test_size)
    return train, test

In [4]:
def apply_drop(train,test,X_train,y_train,X_test,y_test,model):
    
    if 'z' in X_train.columns:
        train = train.drop('z', 1)
        test = test.drop('z', 1)
        X_train = X_train.drop('z', 1)
        X_test = X_test.drop('z', 1)
    
    # this approach is to save the right y values for train and testset
    train_na_free = train.dropna()
    test_na_free = test.dropna()
        
    X_train_use = X_train[X_train.index.isin(train_na_free.index)]
    y_train_use = y_train[y_train.index.isin(train_na_free.index)]    
    
    X_test_use = X_test[X_test.index.isin(test_na_free.index)]
    y_test_use = y_test[y_test.index.isin(test_na_free.index)]
    
    scaler = StandardScaler()
    scaler.fit(X_train_use)
    X_train_use = pd.DataFrame(data=scaler.transform(X_train_use), columns=X_train_use.columns) 
    X_test_use = pd.DataFrame(data=scaler.transform(X_test_use), columns=X_test_use.columns)
        
    mod = model.fit(X_train_use,y_train_use)
    pred = mod.predict(X_test_use)
    
    del scaler
    
    return {'mse': mse(y_test_use,pred),
            'rmse': sqrt(mse(y_test_use,pred)),
            'ev': ev(y_test_use,pred),
            'mae': mae(y_test_use,pred),
            'dif': mse(y_train_use,mod.predict(X_train_use)) - mse(y_test_use,pred)}

In [5]:
# Create imputation function for random, regression and stochastic regression imputation

zip = six.moves.zip
map = six.moves.map

__all__ = [
    'Imputer',
]

def _get_mask(X, value_to_mask):
    """Compute the boolean mask X == missing_values."""
    if value_to_mask == "NaN" or np.isnan(value_to_mask):
        return np.isnan(X)
    else:
        return X == value_to_mask
    
class custom_imputer(Imputer):
       
    def fit(self, X, y = None):
        """Fit the imputer on X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            Input data, where ``n_samples`` is the number of samples and
            ``n_features`` is the number of features.
        Returns
        -------
        self : Imputer
            Returns self.
        """

        # Check parameters
        allowed_strategies = ["mean", "median", "most_frequent", "random", "regression", "stochastic"] 
        if 'z' in X.columns:
            self.remove = True
        else:
            self.remove = False

        if self.strategy not in allowed_strategies:
            raise ValueError("Can only use these strategies: {0} "
                             " got strategy={1}".format(allowed_strategies,
                                                        self.strategy))

        if self.axis not in [0, 1]:
            raise ValueError("Can only impute missing values on axis 0 and 1, "
                             " got axis={0}".format(self.axis))

        # Since two different arrays can be provided in fit(X) and
        # transform(X), the imputation data will be computed in transform()
        # when the imputation is done per sample (i.e., when axis=1).
        if self.axis == 0:
            X = check_array(X, accept_sparse='csc', dtype=np.float64,
                            force_all_finite=False)      

            if sparse.issparse(X):
                self.statistics_ = self._sparse_fit(X, 
                                                    self.strategy,
                                                    self.missing_values,
                                                    self.axis)
            else:
                self.statistics_ = self._dense_fit(X, 
                                                   self.strategy,
                                                   self.missing_values,
                                                   self.axis)
        return self

    def _dense_fit(self, X, strategy, missing_values, axis):
        """Fit the transformer on dense data."""

        X = check_array(X, force_all_finite=False)
        mask = _get_mask(X, missing_values)
        masked_X = ma.masked_array(X, mask=mask)

        # Mean
        if strategy == "mean":
            mean_masked = np.ma.mean(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            mean = np.ma.getdata(mean_masked)
            mean[np.ma.getmask(mean_masked)] = np.nan

            return mean

        # Median
        elif strategy == "median":
            if tuple(int(v) for v in np.__version__.split('.')[:2]) < (1, 5):
                # In old versions of numpy, calling a median on an array
                # containing nans returns nan. This is different is
                # recent versions of numpy, which we want to mimic
                masked_X.mask = np.logical_or(masked_X.mask,
                                              np.isnan(X))
            median_masked = np.ma.median(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            median = np.ma.getdata(median_masked)
            median[np.ma.getmaskarray(median_masked)] = np.nan

            return median

        # Most frequent
        elif strategy == "most_frequent":
            # scipy.stats.mstats.mode cannot be used because it will no work
            # properly if the first element is masked and if its frequency
            # is equal to the frequency of the most frequent valid element
            # See https://github.com/scipy/scipy/issues/2636

            # To be able access the elements by columns
            if axis == 0:
                X = X.transpose()
                mask = mask.transpose()

            most_frequent = np.empty(X.shape[0])

            for i, (row, row_mask) in enumerate(zip(X[:], mask[:])):
                row_mask = np.logical_not(row_mask).astype(np.bool)
                row = row[row_mask]
                most_frequent[i] = _most_frequent(row, np.nan, 0)

            return most_frequent
        
        # Random: in fit: send values that can be selected
        elif strategy == 'random':
            
            not_mask = mask == False
            statistics = []         
            for i in range(X.shape[1]):
                statistics.append(X[:, i][not_mask[:, i]])
            
            return statistics #for each column a vector of possible imputations
                
        elif strategy in ['regression', 'stochastic']:
                                     
            X = X[~np.isnan(X).any(axis=1)] #how to do this by making use of mask/not_mask?            
            statistics = []
            
            lr = LinearRegression()
            for i in range(X.shape[1]):
                dependent_var = X[:, i]
                independent_var = np.delete(X, i, axis = 1)
                lr.fit(independent_var, dependent_var)
                stats_per_column = [lr.intercept_, lr.coef_]
                
                if strategy == 'stochastic':
                    res = dependent_var - lr.predict(independent_var)
                    sd = np.std(res)
                    stats_per_column.append(sd)
                    
                statistics.append(stats_per_column)
                
            # this is the code from strategy == 'mean'
            mean_masked = np.ma.mean(masked_X, axis=axis)
            # Avoid the warning "Warning: converting a masked element to nan."
            mean = np.ma.getdata(mean_masked)
            mean[np.ma.getmask(mean_masked)] = np.nan
            statistics.append(mean)                       
            
            return statistics #for each column the fit of a regression line

    def transform(self, X):
        """Impute all missing values in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            The input data to complete.
        """      

        if self.strategy in ['mean', 'median', 'most_frequent']:
            
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_
                
                if X.shape[1] != statistics.shape[0]:
                    raise ValueError("X has %d features per sample, expected %d"
                                     % (X.shape[1], self.statistics_.shape[0]))

            # Since two different arrays can be provided in fit(X) and
            # transform(X), the imputation data need to be recomputed
            # when the imputation is done per sample
            else:
                X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES,
                                force_all_finite=False, copy=self.copy)

                if sparse.issparse(X):
                    statistics = self._sparse_fit(X, 
                                                  self.strategy,
                                                  self.missing_values,
                                                  self.axis)

                else:
                    statistics = self._dense_fit(X, 
                                                 self.strategy,
                                                 self.missing_values,
                                                 self.axis)

            # Delete the invalid rows/columns        
            invalid_mask = np.isnan(statistics)
            valid_mask = np.logical_not(invalid_mask)
            valid_statistics = statistics[valid_mask]
            valid_statistics_indexes = np.where(valid_mask)[0]
            missing = np.arange(X.shape[not self.axis])[invalid_mask]

            if self.axis == 0 and invalid_mask.any():
                if self.verbose:
                    warnings.warn("Deleting features without "
                                  "observed values: %s" % missing)
                X = X[:, valid_statistics_indexes]
            elif self.axis == 1 and invalid_mask.any():
                raise ValueError("Some rows only contain "
                                 "missing values: %s" % missing)

            # Do actual imputation
            if sparse.issparse(X) and self.missing_values != 0:
                mask = _get_mask(X.data, self.missing_values)
                indexes = np.repeat(np.arange(len(X.indptr) - 1, dtype=np.int),
                                    np.diff(X.indptr))[mask]

                X.data[mask] = valid_statistics[indexes].astype(X.dtype,
                                                            copy=False)
            else:
                if sparse.issparse(X):
                    X = X.toarray()
                mask = _get_mask(X, self.missing_values)
                n_missing = np.sum(mask, axis=self.axis)
                values = np.repeat(valid_statistics, n_missing)

                if self.axis == 0:
                    coordinates = np.where(mask.transpose())[::-1]
                else:
                    coordinates = mask

                X[coordinates] = values
                
            if self.remove:
                X = X[:, :-1]
                          
            return X            
            
        elif self.strategy == 'random':
                    
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_       
                
            #Do actual imputation                  
            if sparse.issparse(X):
                X = X.toarray()
            mask = _get_mask(X, self.missing_values)
            n_missing = np.sum(mask, axis=self.axis)
                     
            values = np.array([])
            for i in range(len(statistics)):
                imputations = np.random.choice(statistics[i],size=n_missing[i],replace=True)
                values = np.append(values, imputations)
            
            if self.axis == 0:
                coordinates = np.where(mask.transpose())[::-1]
            else:
                coordinates = mask

            X[coordinates] = values
            if self.remove:
                X = X[:, :-1]
            
            return X
            
        elif self.strategy in ['regression', 'stochastic']:
                
            if self.axis == 0:
                check_is_fitted(self, 'statistics_')
                X = check_array(X, accept_sparse='csc', dtype=FLOAT_DTYPES, force_all_finite=False, copy=self.copy)
                statistics = self.statistics_  
                mean_values = statistics[-1]
                statistics = statistics[:-1]
                
            #Do actual imputation                  
            if sparse.issparse(X):
                X = X.toarray()
            mask = _get_mask(X, self.missing_values)
            
            values = np.array([])
            for i in range(len(statistics)):
                               
                dependent_var = X[:, i]                          
                independent_var = np.delete(X[mask[:, i]], i, axis=1)
                independent_mask = np.delete(mask[mask[:, i]], i, axis=1)
                
                # this is the code from mean imputation
                # we need complete data to calculate the predictions
                temp_n_missing = np.sum(independent_mask, axis=self.axis)
                temp_mean_values = np.delete(mean_values, i)
                temp_values = np.repeat(temp_mean_values, temp_n_missing)
                coordinates = np.where(independent_mask.transpose())[::-1]
                independent_var[coordinates] = temp_values
                
                imputations = np.dot(independent_var, statistics[i][1]) + statistics[i][0]
                if self.strategy == 'stochastic':
                    noise = np.random.normal(0, statistics[i][2], size = len(imputations))
                    imputations = imputations + noise
                    
                values = np.append(values, imputations)
            
            if self.axis == 0:
                coordinates = np.where(mask.transpose())[::-1]
            else:
                coordinates = mask

            X[coordinates] = values
            if self.remove:
                X = X[:, :-1]
            
            return X  

In [6]:
class custom_pipeline(Pipeline):
    def transform(self, X):
        assert(isinstance(X,pd.DataFrame)), 'X needs to be pd.DataFrame'
        return pd.DataFrame(super().transform(X),columns = X.columns, index=X.index)

def create_custom_pipeline(imputationstrategy, model):
    pipe = custom_pipeline([
        ('imputer',imputationstrategy),
        ('scaler',StandardScaler()),
        ('model',model)])
    return pipe

In [7]:
def apply_mean_median(X_train,y_train,X_test,y_test,model,method='mean'):
    X_train_use = X_train.copy()
    X_test_use = X_test.copy()
    pipe = create_custom_pipeline(imputationstrategy=custom_imputer(strategy=method),model=model)
    mod = pipe.fit(X_train_use,y_train)
    pred = mod.predict(X_test_use)
    return {'mse': mse(y_test,pred),
            'rmse': sqrt(mse(y_test,pred)),
            'ev': ev(y_test,pred),
            'mae': mae(y_test,pred),
            'dif': mse(y_train,mod.predict(X_train_use)) - mse(y_test,pred)}

In [8]:
def apply_custom_imputation(X_train,y_train,X_test,y_test,model,method='random'):
    X_train_use = X_train.copy()
    X_test_use = X_test.copy()
    pipe = create_custom_pipeline(imputationstrategy=custom_imputer(strategy=method),model=model)
    mod = pipe.fit(X_train_use,y_train)
    pred = mod.predict(X_test_use)
    return {'mse': mse(y_test,pred),
            'rmse': sqrt(mse(y_test,pred)),
            'ev': ev(y_test,pred),
            'mae': mae(y_test,pred),
            'dif': mse(y_train,mod.predict(X_train_use)) - mse(y_test,pred)}

## Simulation function

In [9]:
def perform_simulation(df,n_simulations,missing_proportions,output_variable):
    results_mse = []
    results_rmse = []
    results_ev = []
    results_mae = []
    results_dif = []
    for simulation in range(n_simulations):
        print('busy with simulation ', simulation)
                
        train_ind, test_ind = apply_train_test(df)
        train = df.loc[train_ind].copy()
        test = df.loc[test_ind].copy()        
        
        for mechanism in ['MCAR','MARZ','MARX','MNARZ','MNARX']:
            for proportion in missing_proportions:
                
                train_inc = delete_data(train,mechanism,output_variable,missing_row_proportion=proportion).\
                reset_index().drop('index',1)
                test_inc = delete_data(test,mechanism,output_variable,missing_row_proportion=proportion).\
                reset_index().drop('index',1)          
                
                if mechanism == 'MARZ':
                    X_train = train_inc.drop(output_variable,1)
                    y_train = train_inc[output_variable]
                    X_test = test_inc.drop(output_variable,1)
                    y_test = test_inc[output_variable]
                else:
                    X_train = train_inc.drop([output_variable, 'z'],1)
                    y_train = train_inc[output_variable]
                    X_test = test_inc.drop([output_variable, 'z'],1)
                    y_test = test_inc[output_variable]
                
                missing_cells_proportion = round(train.isnull().sum().mean()/len(train),3)
                for name,model in [['lin',LinearRegression()]]: 
                    drop_ = apply_drop(train_inc,test_inc,X_train,y_train,X_test,y_test,model)
                    mean_ = apply_mean_median(X_train,y_train,X_test,y_test,model,method='mean')
                    median_ = apply_mean_median(X_train,y_train,X_test,y_test,model,method='median')
                    random_ = apply_custom_imputation(X_train,y_train,X_test,y_test,model,method='random')   
                    regression_ = apply_custom_imputation(X_train,y_train,X_test,y_test,model,method='regression')   
                    stochastic_ = apply_custom_imputation(X_train,y_train,X_test,y_test,model,method='stochastic')  
                                
                    results_mse.append([simulation,mechanism,proportion,missing_cells_proportion,name,
                                        drop_['mse'],mean_['mse'],median_['mse'],
                                        random_['mse'],regression_['mse'], stochastic_['mse']])
                    results_rmse.append([simulation,mechanism,proportion,missing_cells_proportion,name,
                                        drop_['rmse'],mean_['rmse'],median_['rmse'],
                                        random_['rmse'],regression_['rmse'], stochastic_['rmse']])
                    results_mae.append([simulation,mechanism,proportion,missing_cells_proportion,name,
                                       drop_['mae'],mean_['mae'],median_['mae'],
                                       random_['mae'],regression_['mae'],stochastic_['mae']])
                    results_ev.append([simulation,mechanism,proportion,missing_cells_proportion,name,
                                       drop_['ev'],mean_['ev'],median_['ev'],
                                       random_['ev'],regression_['ev'],stochastic_['ev']])
                    results_dif.append([simulation,mechanism,proportion,missing_cells_proportion,name,
                                       drop_['dif'],mean_['dif'],median_['dif'],
                                       random_['dif'],regression_['dif'],stochastic_['dif']])
    return results_mse, results_rmse, results_ev, results_mae, results_dif

def save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,nsimulations,missing_proportions):
    export_mse = pd.DataFrame(results_mse,columns = ['simulation','missing_type','missing_rows_proportion',
                                                     'missing_cells_proportion','model',
                                                     'drop','mean','median','random','regression','stochastic'])
    export_rmse = pd.DataFrame(results_rmse,columns = ['simulation','missing_type','missing_rows_proportion',
                                                     'missing_cells_proportion','model',
                                                     'drop','mean','median','random','regression','stochastic'])
    export_ev = pd.DataFrame(results_ev,columns = ['simulation','missing_type','missing_rows_proportion',
                                                   'missing_cells_proportion','model',
                                                   'drop','mean','median','random','regression','stochastic'])
    export_mae = pd.DataFrame(results_mae,columns = ['simulation','missing_type','missing_rows_proportion',
                                                   'missing_cells_proportion','model',
                                                   'drop','mean','median','random','regression','stochastic'])
    export_dif = pd.DataFrame(results_dif,columns = ['simulation','missing_type','missing_rows_proportion',
                                                   'missing_cells_proportion','model',
                                                   'drop','mean','median','random','regression','stochastic'])
    export = pd.concat([export_mse, export_rmse, export_ev, export_mae, export_dif])
    rows_one_metric = n_simulations * len(missing_proportions) * 5
    export['evaluation_metric'] = np.append(np.repeat('mse', rows_one_metric),
                                            np.append(np.repeat('rmse', rows_one_metric),
                                                      np.append(np.repeat('ev', rows_one_metric),
                                                                np.append(np.repeat('mae', rows_one_metric),
                                                                          np.repeat('dif', rows_one_metric)))))

    group_dict = {}
    for col in ['drop','mean','median','random', 'regression','stochastic']:
        group_dict[col] = 'mean'
        export[col+'_upper'] = export[col].copy()
        export[col+'_lower'] = export[col].copy()
        group_dict[col+'_lower'] = lambda x: np.percentile(x,25)
        group_dict[col+'_upper'] = lambda x: np.percentile(x,75)
    group_dict['missing_cells_proportion'] = 'mean'

    export = export.groupby(['missing_type','missing_rows_proportion','model','evaluation_metric']).agg(group_dict)
    
    return export    

## Simulation with real dataset: Forest Fires

In [10]:
missing_proportions = np.linspace(0.05,0.55,20)
n_simulations = 3000

In [11]:
df_ = pd.read_csv('../data/forest_fires.txt',sep='\t')
output_variable = 'area'
df_.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [12]:
results_mse, results_rmse, results_ev, results_mae, results_dif = \
perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

busy with simulation  320
busy with simulation  321
busy with simulation  322
busy with simulation  323
busy with simulation  324
busy with simulation  325
busy with simulation  326
busy with simulation  327
busy with simulation  328
busy with simulation  329
busy with simulation  330
busy with simulation  331
busy with simulation  332
busy with simulation  333
busy with simulation  334
busy with simulation  335
busy with simulation  336
busy with simulation  337
busy with simulation  338
busy with simulation  339
busy with simulation  340
busy with simulation  341
busy with simulation  342
busy with simulation  343
busy with simulation  344
busy with simulation  345
busy with simulation  346
busy with simulation  347
busy with simulation  348
busy with simulation  349
busy with simulation  350
busy with simulation  351
busy with simulation  352
busy with simulation  353
busy with simulation  354
busy with simulation  355
busy with simulation  356
busy with simulation  357
busy with si

busy with simulation  636
busy with simulation  637
busy with simulation  638
busy with simulation  639
busy with simulation  640
busy with simulation  641
busy with simulation  642
busy with simulation  643
busy with simulation  644
busy with simulation  645
busy with simulation  646
busy with simulation  647
busy with simulation  648
busy with simulation  649
busy with simulation  650
busy with simulation  651
busy with simulation  652
busy with simulation  653
busy with simulation  654
busy with simulation  655
busy with simulation  656
busy with simulation  657
busy with simulation  658
busy with simulation  659
busy with simulation  660
busy with simulation  661
busy with simulation  662
busy with simulation  663
busy with simulation  664
busy with simulation  665
busy with simulation  666
busy with simulation  667
busy with simulation  668
busy with simulation  669
busy with simulation  670
busy with simulation  671
busy with simulation  672
busy with simulation  673
busy with si

busy with simulation  952
busy with simulation  953
busy with simulation  954
busy with simulation  955
busy with simulation  956
busy with simulation  957
busy with simulation  958
busy with simulation  959
busy with simulation  960
busy with simulation  961
busy with simulation  962
busy with simulation  963
busy with simulation  964
busy with simulation  965
busy with simulation  966
busy with simulation  967
busy with simulation  968
busy with simulation  969
busy with simulation  970
busy with simulation  971
busy with simulation  972
busy with simulation  973
busy with simulation  974
busy with simulation  975
busy with simulation  976
busy with simulation  977
busy with simulation  978
busy with simulation  979
busy with simulation  980
busy with simulation  981
busy with simulation  982
busy with simulation  983
busy with simulation  984
busy with simulation  985
busy with simulation  986
busy with simulation  987
busy with simulation  988
busy with simulation  989
busy with si

In [13]:
export.head()

drop  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif                -416.082132   
                                           ev                   -0.119949   
                                           mae                  20.501991   
                                           mse                4244.595377   
                                           rmse                 59.937091   

                                                               drop_lower  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif               -4357.578931   
                                           ev                   -0.178802   
                                           mae                  19.431900   
                                           mse                1073.595222   
                                           rmse                 32.765763   

                                                               drop_upper  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif                4988.190154   
                                           ev                    0.008270   
                                           mae                  21.890948   
                                           mse                6721.431578   
                                           rmse                 81.984335   

                                                                     mean  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif                -418.706654   
                                           ev                   -0.106623   
                                           mae                  20.440485   
                                           mse                4255.175304   
                                           rmse                 60.369316   

                                                               mean_lower  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif               -4219.967747   
                                           ev                   -0.161849   
                                           mae                  19.421185   
                                           mse                1101.195013   
                                           rmse                 33.184259   

                                                               mean_upper  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif                4866.223800   
                                           ev                    0.008130   
                                           mae                  21.486568   
                                           mse                6525.229031   
                                           rmse                 80.778889   

                                                                   median  \
missing_type missing_rows_proportion model evaluation_metric                
MARX         0.05                    lin   dif                -418.586905   
                                           ev                   -0.106350   
                                           mae                  20.441142   
                                           mse                4255.047499   
                                           rmse                 60.367246   

                                                              median_lower  \
missing_type missing_rows_proportion model evaluation_metric                 
MARX         0.05                    lin   dif                -4220.872556   
                                           ev          

In [14]:
export.to_csv('../results/results_forest_fires.txt',sep='\t')

## Simulation with real dataset: Slump Test

In [15]:
df_ = pd.read_csv('../data/slump_test.txt',sep='\t')
output_variable = 'SLUMP(cm)'
df_.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm)
0,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0
1,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0
2,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0
3,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0
4,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0


In [16]:
results_mse, results_rmse, results_ev, results_mae, results_dif = \
perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_slump_test.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

busy with simulation  320
busy with simulation  321
busy with simulation  322
busy with simulation  323
busy with simulation  324
busy with simulation  325
busy with simulation  326
busy with simulation  327
busy with simulation  328
busy with simulation  329
busy with simulation  330
busy with simulation  331
busy with simulation  332
busy with simulation  333
busy with simulation  334
busy with simulation  335
busy with simulation  336
busy with simulation  337
busy with simulation  338
busy with simulation  339
busy with simulation  340
busy with simulation  341
busy with simulation  342
busy with simulation  343
busy with simulation  344
busy with simulation  345
busy with simulation  346
busy with simulation  347
busy with simulation  348
busy with simulation  349
busy with simulation  350
busy with simulation  351
busy with simulation  352
busy with simulation  353
busy with simulation  354
busy with simulation  355
busy with simulation  356
busy with simulation  357
busy with si

busy with simulation  636
busy with simulation  637
busy with simulation  638
busy with simulation  639
busy with simulation  640
busy with simulation  641
busy with simulation  642
busy with simulation  643
busy with simulation  644
busy with simulation  645
busy with simulation  646
busy with simulation  647
busy with simulation  648
busy with simulation  649
busy with simulation  650
busy with simulation  651
busy with simulation  652
busy with simulation  653
busy with simulation  654
busy with simulation  655
busy with simulation  656
busy with simulation  657
busy with simulation  658
busy with simulation  659
busy with simulation  660
busy with simulation  661
busy with simulation  662
busy with simulation  663
busy with simulation  664
busy with simulation  665
busy with simulation  666
busy with simulation  667
busy with simulation  668
busy with simulation  669
busy with simulation  670
busy with simulation  671
busy with simulation  672
busy with simulation  673
busy with si

busy with simulation  952
busy with simulation  953
busy with simulation  954
busy with simulation  955
busy with simulation  956
busy with simulation  957
busy with simulation  958
busy with simulation  959
busy with simulation  960
busy with simulation  961
busy with simulation  962
busy with simulation  963
busy with simulation  964
busy with simulation  965
busy with simulation  966
busy with simulation  967
busy with simulation  968
busy with simulation  969
busy with simulation  970
busy with simulation  971
busy with simulation  972
busy with simulation  973
busy with simulation  974
busy with simulation  975
busy with simulation  976
busy with simulation  977
busy with simulation  978
busy with simulation  979
busy with simulation  980
busy with simulation  981
busy with simulation  982
busy with simulation  983
busy with simulation  984
busy with simulation  985
busy with simulation  986
busy with simulation  987
busy with simulation  988
busy with simulation  989
busy with si

## Simulation with real dataset: Red Wine Quality

In [ ]:
df_ = pd.read_csv('../data/red_wine_quality.txt',sep='\t')
output_variable = 'quality'
df_.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
results_mse, results_rmse, results_ev, results_mae, results_dif = \
perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_red_wine_quality.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

busy with simulation  320
busy with simulation  321
busy with simulation  322
busy with simulation  323
busy with simulation  324
busy with simulation  325
busy with simulation  326
busy with simulation  327
busy with simulation  328
busy with simulation  329
busy with simulation  330
busy with simulation  331
busy with simulation  332
busy with simulation  333
busy with simulation  334
busy with simulation  335
busy with simulation  336
busy with simulation  337
busy with simulation  338
busy with simulation  339
busy with simulation  340
busy with simulation  341
busy with simulation  342
busy with simulation  343
busy with simulation  344
busy with simulation  345
busy with simulation  346
busy with simulation  347
busy with simulation  348
busy with simulation  349
busy with simulation  350
busy with simulation  351
busy with simulation  352
busy with simulation  353
busy with simulation  354
busy with simulation  355
busy with simulation  356
busy with simulation  357
busy with si

busy with simulation  636
busy with simulation  637
busy with simulation  638
busy with simulation  639
busy with simulation  640
busy with simulation  641
busy with simulation  642
busy with simulation  643
busy with simulation  644
busy with simulation  645
busy with simulation  646
busy with simulation  647
busy with simulation  648
busy with simulation  649
busy with simulation  650
busy with simulation  651
busy with simulation  652
busy with simulation  653
busy with simulation  654
busy with simulation  655
busy with simulation  656
busy with simulation  657
busy with simulation  658
busy with simulation  659
busy with simulation  660
busy with simulation  661
busy with simulation  662
busy with simulation  663
busy with simulation  664
busy with simulation  665
busy with simulation  666
busy with simulation  667
busy with simulation  668
busy with simulation  669
busy with simulation  670
busy with simulation  671
busy with simulation  672
busy with simulation  673
busy with si

busy with simulation  952
busy with simulation  953
busy with simulation  954
busy with simulation  955
busy with simulation  956
busy with simulation  957
busy with simulation  958
busy with simulation  959
busy with simulation  960
busy with simulation  961
busy with simulation  962
busy with simulation  963
busy with simulation  964
busy with simulation  965
busy with simulation  966
busy with simulation  967
busy with simulation  968
busy with simulation  969
busy with simulation  970
busy with simulation  971
busy with simulation  972
busy with simulation  973
busy with simulation  974
busy with simulation  975
busy with simulation  976
busy with simulation  977
busy with simulation  978
busy with simulation  979
busy with simulation  980
busy with simulation  981
busy with simulation  982
busy with simulation  983
busy with simulation  984
busy with simulation  985
busy with simulation  986
busy with simulation  987
busy with simulation  988
busy with simulation  989
busy with si

## Simulation with real dataset: School Alcohol Consumption

In [11]:
df_ = pd.read_csv('../data/school_alcohol_consumption.txt',sep='\t')
output_variable = 'Dalc'
df_.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10


In [12]:
results_mse, results_rmse, results_ev, results_mae, results_dif = \
perform_simulation(df_,n_simulations,missing_proportions,output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_school_alcohol_consumption.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
busy with simulation  25
busy with simulation  26
busy with simulation  27
busy with simulation  28
busy with simulation  29
busy with simulation  30
busy with simulation  31
busy with simulation  32
busy with simulation  33
busy with simulation  34
busy with simulation  35
busy with simulation  36
busy with simulation  37
busy with simulation  38
busy with simulation  39
busy with 

busy with simulation  320
busy with simulation  321
busy with simulation  322
busy with simulation  323
busy with simulation  324
busy with simulation  325
busy with simulation  326
busy with simulation  327
busy with simulation  328
busy with simulation  329
busy with simulation  330
busy with simulation  331
busy with simulation  332
busy with simulation  333
busy with simulation  334
busy with simulation  335
busy with simulation  336
busy with simulation  337
busy with simulation  338
busy with simulation  339
busy with simulation  340
busy with simulation  341
busy with simulation  342
busy with simulation  343
busy with simulation  344
busy with simulation  345
busy with simulation  346
busy with simulation  347
busy with simulation  348
busy with simulation  349
busy with simulation  350
busy with simulation  351
busy with simulation  352
busy with simulation  353
busy with simulation  354
busy with simulation  355
busy with simulation  356
busy with simulation  357
busy with si

busy with simulation  636
busy with simulation  637
busy with simulation  638
busy with simulation  639
busy with simulation  640
busy with simulation  641
busy with simulation  642
busy with simulation  643
busy with simulation  644
busy with simulation  645
busy with simulation  646
busy with simulation  647
busy with simulation  648
busy with simulation  649
busy with simulation  650
busy with simulation  651
busy with simulation  652
busy with simulation  653
busy with simulation  654
busy with simulation  655
busy with simulation  656
busy with simulation  657
busy with simulation  658
busy with simulation  659
busy with simulation  660
busy with simulation  661
busy with simulation  662
busy with simulation  663
busy with simulation  664
busy with simulation  665
busy with simulation  666
busy with simulation  667
busy with simulation  668
busy with simulation  669
busy with simulation  670
busy with simulation  671
busy with simulation  672
busy with simulation  673
busy with si

busy with simulation  952
busy with simulation  953
busy with simulation  954
busy with simulation  955
busy with simulation  956
busy with simulation  957
busy with simulation  958
busy with simulation  959
busy with simulation  960
busy with simulation  961
busy with simulation  962
busy with simulation  963
busy with simulation  964
busy with simulation  965
busy with simulation  966
busy with simulation  967
busy with simulation  968
busy with simulation  969
busy with simulation  970
busy with simulation  971
busy with simulation  972
busy with simulation  973
busy with simulation  974
busy with simulation  975
busy with simulation  976
busy with simulation  977
busy with simulation  978
busy with simulation  979
busy with simulation  980
busy with simulation  981
busy with simulation  982
busy with simulation  983
busy with simulation  984
busy with simulation  985
busy with simulation  986
busy with simulation  987
busy with simulation  988
busy with simulation  989
busy with si

## Simulation with custom dataset

In [13]:
n_simulations = 100

In [14]:
#custom_dataset_poor_small
df_ = pd.read_csv('../data/custom_dataset_poor_little.txt',sep='\t')
output_variable = 'y'
results_mse, results_rmse, results_ev, results_mae, results_dif = perform_simulation(df_,
                                                                        n_simulations,
                                                                        missing_proportions,
                                                                        output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_custom_dataset_poor_little.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24


In [15]:
#custom_dataset_poor_large
df_ = pd.read_csv('../data/custom_dataset_poor_much.txt',sep='\t')
output_variable = 'y'
results_mse, results_rmse, results_ev, results_mae, results_dif = perform_simulation(df_,
                                                                        n_simulations,
                                                                        missing_proportions,
                                                                        output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_custom_dataset_poor_much.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24


In [16]:
#custom_dataset_rich_small
df_ = pd.read_csv('../data/custom_dataset_rich_little.txt',sep='\t')
output_variable = 'y'
results_mse, results_rmse, results_ev, results_mae, results_dif = perform_simulation(df_,
                                                                        n_simulations,
                                                                        missing_proportions,
                                                                        output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_custom_dataset_rich_little.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24


In [17]:
#custom_dataset_rich_large
df_ = pd.read_csv('../data/custom_dataset_rich_much.txt',sep='\t')
output_variable = 'y'
results_mse, results_rmse, results_ev, results_mae, results_dif = perform_simulation(df_,
                                                                        n_simulations,
                                                                        missing_proportions,
                                                                        output_variable)
export = save_results(results_mse, results_rmse, results_ev, results_mae, results_dif,n_simulations,missing_proportions)
export.to_csv('../results/results_custom_dataset_rich_much.txt',sep='\t')

busy with simulation  0
busy with simulation  1
busy with simulation  2
busy with simulation  3
busy with simulation  4
busy with simulation  5
busy with simulation  6
busy with simulation  7
busy with simulation  8
busy with simulation  9
busy with simulation  10
busy with simulation  11
busy with simulation  12
busy with simulation  13
busy with simulation  14
busy with simulation  15
busy with simulation  16
busy with simulation  17
busy with simulation  18
busy with simulation  19
busy with simulation  20
busy with simulation  21
busy with simulation  22
busy with simulation  23
busy with simulation  24
